In [2]:
import re
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import pandas as pd


options = webdriver.ChromeOptions()
#options.add_argument("headless")

In [3]:
driver_path = 'C:/chromedriver_win32/chromedriver.exe' 

In [28]:
def donga(query,driver_path):
    domain = '동아일보'
    driver = webdriver.Chrome(executable_path=driver_path, options=options)  # for Windows
    url_domain = 'https://search.naver.com/search.naver?sm=tab_hty.top&where=news&query='
    news_domain ='&sm=tab_opt&sort=0&photo=0&field=0&pd=0&ds=&de=&docid=&related=0&mynews=1&office_type=1&office_section_code=1&news_office_checked=1020&nso=&is_sug_officeid=0' 
    url = url_domain + query + news_domain
    driver.get(url)
    soup = BeautifulSoup(driver.page_source,'html.parser')
 
    # 기사 url을 list형태로 반환
    news_list = soup.select('#main_pack > section > div > div.group_news > ul')
    url_list = []
    news_list = news_list[0].find_all('li')
    
    for url in news_list:
        url_list.append(str(url.find_all('a')[0].get('data-url')))
    driver.close()
    
    data_list = []
    head_list = []
    for url in url_list:
        driver = webdriver.Chrome(executable_path=driver_path,options=options)  # for Windows
        time.sleep(2)
        driver.get(url)
        time.sleep(2)
        # try문 안에서 각자 언론사에 맞게 수정
        try:
            soup = BeautifulSoup(driver.page_source,'html.parser')
            head_list.append(soup.select_one('#container > div.article_title > h1').text)
            tags = soup.select_one('#content > div > div.article_txt').text
            tags=clean_text(tags)
            data_list.append(tags)
        except:
            print('None_type')
        driver.close()
        
    output = pd.DataFrame({'언론사' : [domain]*len(head_list),
                          '제목' :  head_list,
                          '내용' : data_list})
    return output

donga = donga('삼성전자 슬로건',driver_path)

None_type


In [27]:
def clean_text(text):
    cleaned_text = re.sub('[a-zA-Z]', '', text)
    cleaned_text = re.sub('[\{\}\[\]\/?.,;:|\)*~`!^\-_+<>@\#$%&\\\=\(\'\"]',
                          '', cleaned_text)
    cleaned_text = ' '.join(cleaned_text.split())
    return cleaned_text

In [30]:
donga.to_csv('동아일보.csv',encoding='CP949',index=None)
dong=pd.read_csv('동아일보.csv',encoding='CP949')

In [31]:
dong

,언론사,제목,내용
0,동아일보,“삼성 모든 가전의 ‘엔진’ 평생보증 자신있다”,카테크이기수 삼성전자 생활가전사업부 부사장 5일 이기수 삼성전자 생활가전사업부 개발...
1,동아일보,삼성전자 맞춤형 가전 ‘비스포크’ 인기몰이,국내 냉장고 매출 65가 비스포크 식기세척기인덕션 매출도 급증 삼성전자의 비스포크 ...
2,동아일보,"삼성전자, 갤러리아 광교점에 체험 스튜디오",삼성전자가 경기 수원시 갤러리아백화점 광교점에 ‘나답게 스튜디오’를 마련했다 나답게...
3,동아일보,‘이제는 가전을 나답게’ 삼성 새 슬로건 공개,‘이제는 가전을 나답게’ 삼성전자가 가전제품 마케팅에 사용될 새 슬로건을 17일 공...
4,동아일보,"[기업열전] 대우전자 DNA 품은 종합가전기업, 루컴즈전자",서울 서초구 방배동의 루컴즈 빌딩 출처루컴즈전자루컴즈전자 출처루컴즈전자1990년대 ...
5,동아일보,삼성전자 ‘3인 체제’ 안정 속 파격 인사…첫 가전 출신 사장도,삼성전자가 2일 사장 승진 3명 위촉 업무 변경 2명 등 총 5명 규모의 2021년...
6,동아일보,[횡설수설/최영해]좋은 슬로건 나쁜 슬로건,지난해 여름 미국 샌프란시스코에 출장을 갔을 때다 휴대전화에 문제가 생겨 통신회사 ...
7,동아일보,[품질경쟁력 우수기업]삼성전자 “Live SMART” 고객의 삶에 새로운 가치를 심다,거리에서 사무실에서 집에서 커피숍에서 우리가 마주치는 거의 모든 사람이 손에 들고 ...
8,동아일보,"삼성전자, 밀레니얼 취향 저격 마케팅으로 혼수가전 매출 65% ↑",삼성전자는 올 상반기 혼수가전 매출이 전년 동기 대비 약 65증가했다고 15일 밝혔...
